### 서버 용량 제한으로 csv 파일에 있는 두 모댈의 예측 prob 값으로 앙상블

In [40]:
import pandas as pd
import numpy as np
import ast

In [50]:
qwen_probs_df = pd.read_csv("./qwen3_probs_v1.csv")
as_probs_df = pd.read_csv("./ax_probs_v1.csv")

In [51]:
as_probs_df.iloc[0]['digit_probs']

'[0.003936690278351307, 0.0008381697698496282, 0.0004927345435135067, 0.0008783943485468626, 0.9938540458679199]'

In [52]:
import ast

# 1. 각 모델의 예측값 계산 (문자열 -> 리스트 -> 넘파이 배열 -> argmax)
qwen_probs_df['pred'] = qwen_probs_df['digit_probs'].apply(
    lambda x: np.argmax(np.array(ast.literal_eval(x), dtype=float)) + 1
)
as_probs_df['pred'] = as_probs_df['digit_probs'].apply(
    lambda x: np.argmax(np.array(ast.literal_eval(x), dtype=float)) + 1
)

# 2. 두 모델의 예측이 다른 샘플 추출
diff_mask = qwen_probs_df['pred'] != as_probs_df['pred']

# 3. 비교용 데이터프레임 (예측값 숫자를 정확히 담기)
comparison_df = pd.DataFrame({
    'id': qwen_probs_df.loc[diff_mask, 'id'],
    'qwen_pred': qwen_probs_df.loc[diff_mask, 'pred'],
    'ax_pred': as_probs_df.loc[diff_mask, 'pred'],
    'qwen_probs_raw': qwen_probs_df.loc[diff_mask, 'digit_probs'],
    'ax_probs_raw': as_probs_df.loc[diff_mask, 'digit_probs']
})

# 4. 리포트 출력
total = len(qwen_probs_df)
diff_count = len(comparison_df)
print(f"=== 모델 예측 차이 분석 리포트 ===")
print(f"전체 샘플 수: {total}")
print(f"엇갈린 샘플  : {diff_count} ({diff_count/total*100:.2f}%)")

=== 모델 예측 차이 분석 리포트 ===
전체 샘플 수: 869
엇갈린 샘플  : 223 (25.66%)


In [53]:
# 2. 앙상블 계산
ensemble_results = []

# 가중치 설정 (합이 1이 되도록 설정하는 것이 일반적입니다)
w_qwen = 0.6  # Qwen 비중
w_ax = 0.4    # AX 비중

ensemble_results = []

for p1, p2 in zip(qwen_probs_df['digit_probs'], as_probs_df['digit_probs']):
    p1 = ast.literal_eval(p1)
    p2 = ast.literal_eval(p2)
    
    arr1 = np.array(p1, dtype=float)
    arr2 = np.array(p2, dtype=float)
    
    # [수정] 가중치 적용 (Weighted Soft Voting)
    weighted_prob = (arr1 * w_qwen) + (arr2 * w_ax)
    
    # 가장 높은 확률의 인덱스 + 1
    final_answer = np.argmax(weighted_prob) + 1
    ensemble_results.append(final_answer)

print(f"✅ 가중치 {w_qwen}:{w_ax} 적용 앙상블 완료")

# 3. 최종 결과 저장
submission = pd.DataFrame({
    'id': qwen_probs_df['id'],
    'answer': ensemble_results
})

submission.to_csv('submission_ensemble_v1.csv', index=False)
print("✅ 앙상블 완료: submission_ensemble_V1.csv 저장됨")

✅ 가중치 0.6:0.4 적용 앙상블 완료
✅ 앙상블 완료: submission_ensemble_V1.csv 저장됨


In [54]:

# 1. 앙상블 결과 데이터프레임 생성 (이전 단계 결과물 활용)
submission_df = pd.DataFrame({
    'id': qwen_probs_df['id'],
    'qwen_ans': qwen_probs_df['pred'],
    'ax_ans': as_probs_df['pred'],
    'ensemble_ans': ensemble_results
})

# 2. 일치 여부 체크 컬럼 추가
# Qwen과 AX가 의견이 갈렸을 때, 앙상블이 누구 편을 들었는지 확인
def check_who_won(row):
    if row['qwen_ans'] == row['ax_ans']:
        return "Both Agree"
    elif row['ensemble_ans'] == row['qwen_ans']:
        return "Ensemble followed Qwen"
    elif row['ensemble_ans'] == row['ax_ans']:
        return "Ensemble followed AX"
    else:
        return "Ensemble chose something else" # 매우 드문 경우 (확률 합산의 묘미)

submission_df['analysis'] = submission_df.apply(check_who_won, axis=1)

# 3. 통계 출력
summary = submission_df['analysis'].value_counts()
total = len(submission_df)

print("=== 앙상블 의사결정 분석 ===")
for status, count in summary.items():
    print(f"- {status}: {count}개 ({count/total*100:.2f}%)")
print("============================")

# 4. 앙상블로 인해 정답이 바뀐(엇갈린) 샘플들만 출력
disagree_samples = submission_df[submission_df['analysis'] != "Both Agree"]
print("\n[의견이 엇갈렸던 샘플 분석 (상위 10개)]")
print(disagree_samples[['id', 'qwen_ans', 'ax_ans', 'ensemble_ans', 'analysis']].head(10))

=== 앙상블 의사결정 분석 ===
- Both Agree: 646개 (74.34%)
- Ensemble followed Qwen: 176개 (20.25%)
- Ensemble followed AX: 47개 (5.41%)

[의견이 엇갈렸던 샘플 분석 (상위 10개)]
                       id  qwen_ans  ax_ans  ensemble_ans  \
7    generation-for-nlp-7         2       3             2   
11  generation-for-nlp-11         4       1             4   
16  generation-for-nlp-16         3       1             3   
17  generation-for-nlp-17         3       5             3   
22  generation-for-nlp-22         5       2             2   
23  generation-for-nlp-23         3       1             3   
27  generation-for-nlp-27         4       2             4   
31  generation-for-nlp-31         1       2             2   
33  generation-for-nlp-33         3       2             3   
34  generation-for-nlp-34         5       4             5   

                  analysis  
7   Ensemble followed Qwen  
11  Ensemble followed Qwen  
16  Ensemble followed Qwen  
17  Ensemble followed Qwen  
22    Ensemble followed AX  
23 

In [55]:
import pandas as pd
import numpy as np

# CSV 파일 로드
df1 = pd.read_csv('submission_ensemble.csv')
df2 = pd.read_csv('submission_ensemble_v1.csv')

# 병합 (id 기준)
merged = df1.merge(df2, on='id', suffixes=('_v1', '_v2'))

print("=== 기본 정보 ===")
print(f"전체 샘플 수: {len(merged)}")
print()

# 1. 일치율
same_predictions = (merged['answer_v1'] == merged['answer_v2']).sum()
different_predictions = len(merged) - same_predictions

print("=== 예측 일치율 ===")
print(f"동일한 예측: {same_predictions:,}개 ({same_predictions/len(merged)*100:.2f}%)")
print(f"다른 예측:   {different_predictions:,}개 ({different_predictions/len(merged)*100:.2f}%)")
print()

# 2. 답변 분포
print("=== 각 모델의 답변 분포 ===")
print("\nsubmission_ensemble.csv:")
print(df1['answer'].value_counts().sort_index())
print(f"\nsubmission_ensemble_v1.csv:")
print(df2['answer'].value_counts().sort_index())
print()

# 3. 차이나는 샘플들 상세 분석
if different_predictions > 0:
    diff_samples = merged[merged['answer_v1'] != merged['answer_v2']]
    
    print(f"=== 차이나는 샘플 상세 ({len(diff_samples)}개) ===")
    print(diff_samples.head(20))
    print()
    
    # 4. 변화 패턴 분석
    print("=== 예측 변화 패턴 (v1 → v2) ===")
    transition = diff_samples.groupby(['answer_v1', 'answer_v2']).size().reset_index(name='count')
    transition = transition.sort_values('count', ascending=False)
    
    for _, row in transition.iterrows():
        print(f"{row['answer_v1']} → {row['answer_v2']}: {row['count']:3d}개")
    print()
    
    # 5. Confusion Matrix 스타일
    print("=== Confusion Matrix (v1 vs v2) ===")
    confusion = pd.crosstab(
        merged['answer_v1'], 
        merged['answer_v2'],
        rownames=['v1'],
        colnames=['v2']
    )
    print(confusion)
    print()

# 6. 통계 요약
print("=== 통계 요약 ===")
print(f"평균 차이 (절대값): {np.abs(merged['answer_v1'] - merged['answer_v2']).mean():.3f}")
print(f"최대 차이: {np.abs(merged['answer_v1'] - merged['answer_v2']).max()}")
print(f"차이 표준편차: {np.abs(merged['answer_v1'] - merged['answer_v2']).std():.3f}")

# 7. 차이나는 샘플 저장
if different_predictions > 0:
    diff_samples.to_csv('difference_analysis.csv', index=False)
    print(f"\n차이나는 샘플을 'difference_analysis.csv'에 저장했습니다.")

=== 기본 정보 ===
전체 샘플 수: 869

=== 예측 일치율 ===
동일한 예측: 869개 (100.00%)
다른 예측:   0개 (0.00%)

=== 각 모델의 답변 분포 ===

submission_ensemble.csv:
answer
1    231
2    200
3    168
4    157
5    113
Name: count, dtype: int64

submission_ensemble_v1.csv:
answer
1    231
2    200
3    168
4    157
5    113
Name: count, dtype: int64

=== 통계 요약 ===
평균 차이 (절대값): 0.000
최대 차이: 0
차이 표준편차: 0.000
